<a href="https://colab.research.google.com/github/Sakib56/TTDS-G35-CW3/blob/main/back_end/python/vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Uncomment and run the following cells if you work on GCP. Change runtime type to GPU.

In [1]:
pip install torch==1.8.1 transformers==3.3.1 sentence-transformers==0.3.8 pandas==1.1.2 faiss-cpu==1.6.1 numpy==1.19.2 folium==0.2.1 streamlit==0.62.0

     |████████████████████████████████| 804.1 MB 2.8 kB/s 
     |████████████████████████████████| 1.1 MB 11.9 MB/s 
     |████████████████████████████████| 66 kB 5.1 MB/s 
     |████████████████████████████████| 10.5 MB 9.0 MB/s 
     |████████████████████████████████| 7.1 MB 20.7 MB/s 
     |████████████████████████████████| 14.5 MB 14.2 MB/s 
     |████████████████████████████████| 69 kB 10.8 MB/s 
     |████████████████████████████████| 7.1 MB 26.9 MB/s 
     |████████████████████████████████| 1.2 MB 100.0 MB/s 
     |████████████████████████████████| 895 kB 92.1 MB/s 
     |████████████████████████████████| 3.0 MB 75.5 MB/s 
     |████████████████████████████████| 4.3 MB 71.8 MB/s 
     |████████████████████████████████| 8.6 MB 60.0 MB/s 
     |████████████████████████████████| 132 kB 72.5 MB/s 
     |████████████████████████████████| 76 kB 7.2 MB/s 
     |████████████████████████████████| 111 kB 102.1 MB/s 
     |████████████████████████████████| 138 kB 81.8 MB/s 
     |█████████



---
This is mounting my (Kenza) drive to the collab notebook. I stored the wikidata there.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Before we begin, make sure you restart (not factory reset) the runtime so that the relevant packages are used

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

import xml

from nltk.stem import PorterStemmer
import re
ps = PorterStemmer()

In [4]:
def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level 
    DistilBERT model and finds similar vectors using FAISS.
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    # query = ps.stem(query)
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I


def id2details(I):
    """Returns the paper titles based on the paper index."""
    return [worker.pids[str(idx)] for idx in I[0]]

The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

In [5]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

100%|██████████| 245M/245M [00:30<00:00, 8.06MB/s]


cuda:0


In [20]:
class wikiHandler(xml.sax.ContentHandler):

    def __init__(self, searchClass):
        self.tag = ""
        self.pid = None
        self.title = ""
        self.text = ""
        self.searcher = searchClass
        #self.executor = ProcessPoolExecutor(max_workers=1)
        self.progress = tqdm(total=70000000)

    def ended(self):
        self.progress.close()
        self.executor.shutdown()

    def startElement(self, tag, argument):
        self.tag = tag

    def characters(self, content):
        if self.tag == "id" and not content.isspace() and self.pid == None:
            self.pid = content
        if self.tag == "title":
            self.title += content
        if self.tag == "text":
            self.text += content

    def endElement(self, tag):
        self.progress.update(1)
        if tag == "page":
            self.searcher.perpage({"pid":self.pid, "title":self.title, "text":self.text})
            self.pid = None
            self.title = ""
            self.text = ""

In [29]:
# Convert abstracts to vectors
class encoder():

  def __init__(self):
    self.embeddings = []
    self.partials = []
    self.partial_ids={}
    self.pids = {}
    self.quantizer = faiss.IndexFlatL2(768)
    self.nlist = 100
    self.index = faiss.IndexIVFFlat(self.quantizer, 768, self.nlist)
    self.count = 0

  def perpage(self, text):
    self.pids[text["pid"]] = text["title"]
    self.partial_ids[text["pid"]] = text["title"]
    encoding = model.encode(text["text"])
    self.embeddings.append(encoding)
    self.partials.append(encoding)
    self.count += 1
    print(self.count)
    if self.count % 4000 == 0:
      print("training index...")
      self.partials = np.array([embedding for embedding in self.partials]).astype("float32")
      self.index.train(self.partials)
      index = faiss.IndexIDMap(self.index)
      index.add_with_ids(self.partials, np.array(list(self.partial_ids.keys())).astype('int64'))
      with open("/content/faiss_index.pickle", "ab") as h:
        pickle.dump(faiss.serialize_index(index), h)
      self.partials = []
      self.partial_ids = {}
      index.reset()



worker = encoder()
parser = xml.sax.make_parser()  
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
handler = wikiHandler(worker)
parser.setContentHandler(handler)

##### INPUT PATH TO LONG XML!!!!!
parser.parse("/content/drive/MyDrive/TTDS/wikidata_notsoshort.xml")








  0%|          | 0/70000000 [00:00<?, ?it/s]

1









  0%|          | 72/70000000 [00:01<369:35:16, 52.61it/s]






  0%|          | 246/70000000 [00:01<93:36:04, 207.74it/s]

2
3
4
5
6
7
8
9
10
11









  0%|          | 332/70000000 [00:05<387:48:36, 50.14it/s]

...








  0%|          | 1738/70000000 [00:44<868:38:13, 22.38it/s] 

98









  0%|          | 1750/70000000 [00:45<978:48:40, 19.86it/s]






  0%|          | 1759/70000000 [00:53<3059:16:27,  6.36it/s]

99


KeyboardInterrupt: ignored

In [ ]:
# Should the entire thing run properly, try running this to make sure index is 100% correct:
quantizer = faiss.IndexFlatL2(768)
nlist = 100
index = faiss.IndexIVFFlat(quantizer, 768, nlist)
worker.embeddings = np.array([embedding for embedding in worker.embeddings]).astype("float32")
index.train(worker.embeddings)
index = faiss.IndexIDMap(index)
index.add_with_ids(worker.embeddings, np.array(list(worker.pids.keys())).astype('int64'))
with open("/content/final_faiss_index.pickle", "ab") as h:
  pickle.dump(faiss.serialize_index(index), h)

In [ ]:
print(f'Number of articles processed: {len(worker.embeddings)}')

Shape of the vectorised abstract: 4171



## Putting all together

So far, we've built a Faiss index using the wikidata text vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the stemmed query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

IDEA: Use the Answer of the Question Answering option as the input query for vector search or let the user write a query for vector search or both.


In [ ]:
user_query = """Artificial Intelligence"""

In [ ]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [183.08160400390625, 195.53298950195312, 221.16964721679688, 243.65826416015625, 245.3956756591797, 246.18502807617188, 250.58746337890625, 253.0164794921875, 255.68038940429688, 256.96875]

MAG paper IDs: [1164, 2862, 2142, 6596, 5783, 5309, 1456, 6216, 5311, 1349]


In [ ]:
# Fetching the paper titles based on their index
id2details(I)

['Artificial intelligence\n    ',
 'AI-complete\n    ',
 'List of artificial intelligence projects\n    ',
 'Computer vision\n    ',
 'Computer program\n    ',
 'Software\n    ',
 'AWK\n    ',
 'Chinese room\n    ',
 'Computer programming\n    ',
 'Atanasoff–Berry computer\n    ']